In [ ]:
%env OPENAI_API_KEY="keyhere"

In [ ]:
#!/bin/bash

helm upgrade --install argo-rollouts argo-rollouts \
  --repo https://argoproj.github.io/argo-helm \
  --version 2.37.6 \
  --namespace argo-rollouts \
  --create-namespace \
  --wait

kubectl apply -f- <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: rollouts-demo
  labels:
    app: rollouts-demo
spec:
  replicas: 4
  selector:
    matchLabels:
      app: rollouts-demo
  template:
    metadata:
      labels:
        app: rollouts-demo
    spec:
      containers:
      - name: rollouts-demo
        image: argoproj/rollouts-demo:blue
        ports:
        - name: http
          containerPort: 8080
          protocol: TCP
  strategy:
    type: RollingUpdate
    rollingUpdate:
      maxSurge: 1
      maxUnavailable: 1
---
apiVersion: v1
kind: Service
metadata:
  name: rollouts-demo
  labels:
    app: rollouts-demo
spec:
  ports:
  - port: 80
    targetPort: http
    protocol: TCP
    name: http
  selector:
    app: rollouts-demo
apiVersion: gateway.networking.k8s.io/v1
kind: Gateway
metadata:
  name: gateway
spec:
  gatewayClassName: istio
  listeners:
  - name: default
    port: 80
    protocol: HTTP
    allowedRoutes:
      namespaces:
        from: All
---
kind: HTTPRoute
apiVersion: gateway.networking.k8s.io/v1beta1
metadata:
  name: rollouts-demo
spec:
  parentRefs:
    - name: gateway
  rules:
  - matches:
    - path:
        type: PathPrefix
        value: /  
    backendRefs:
    - name: rollouts-demo
      kind: Service
      port: 80
EOF

In [1]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

from kagent.tools.argo import (
    CreateRolloutResource,
    GetRollout,
    PauseRollout,
    PromoteRollout,
    SetRolloutImage,
    StatusRollout,
    VerifyInstall,
    GenerateResource,
)
from kagent.tools.k8s import GetPods, GetServices, GetResources, ApplyManifest

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

argo_agent = AssistantAgent(
    "argo_agent",
    description="An agent for Argo Rollouts operations",
    tools=[
        CreateRolloutResource(),
        GetRollout(),
        PauseRollout(),
        PromoteRollout(),
        SetRolloutImage(),
        StatusRollout(),
        VerifyInstall(),
        GenerateResource(),
    ],
    model_client=model_client,
    system_message="""
    You are an Argo Rollouts agent, specialized in managing and automating the deployment of applications using Argo Rollouts 
    in a Kubernetes cluster. Your expertise includes creating, updating, and monitoring rollouts, as well as handling traffic
    management and canary deployments. Your job is to assist with Argo Rollouts operations efficiently and effectively. 
    DO NOT MAKE UP ADDITIONAL AND UNNECESSARY SUBTASKS.
    """,
)

k8s_agent = AssistantAgent(
    "k8s_agent",
    description="An agent for k8s operations",
    tools=[GetPods(), GetServices(), GetResources(), ApplyManifest()],
    model_client=model_client,
    system_message="""
    You are a k8s agent. You know how to interact with a Kubernetes cluster.

    If you don't have any explicit tasks left to complete, return TERMINATE.
    """,
)

team = RoundRobinGroupChat(
    max_turns=3,
    participants=[k8s_agent, argo_agent],
    termination_condition=TextMentionTermination(text="TERMINATE"),
)

# task = "Create an Argo Rollout to deploy a new version of the demo application with the color purple using the Kubernetes Gateway API in my cluster."
# task = "Check if there are any argo rollout in the cluster in the process of promotion?"
task = "Use the Kubernetes Gateway API and Argo Rollouts to create rollout resources for the canary and stable services for the demo application in my cluster."

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------
Use the Kubernetes Gateway API and Argo Rollouts to create rollout resources for the canary and stable services for the demo application in my cluster.
---------- k8s_agent ----------
To create rollout resources for the canary and stable services using Kubernetes Gateway API and Argo Rollouts, you will need to apply manifests that define these resources. You will typically need to define a Rollout manifest that includes strategies for canary deployments as well as integration with the Gateway API to manage traffic splitting between versions.

Here's a high-level outline of what needs to be done:

1. **Create Rollout Manifests**: You'll need to create one or more YAML manifests that describe the rollout resources. This includes specifying strategies such as canary or blue-green, and configuration for workload, canary steps, traffic routing, etc.

2. **Use Gateway API for traffic management**: Integrate the rollout with the Gateway API to control traffic splitt

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Use the Kubernetes Gateway API and Argo Rollouts to create rollout resources for the canary and stable services for the demo application in my cluster.', type='TextMessage'), TextMessage(source='k8s_agent', models_usage=RequestUsage(prompt_tokens=448, completion_tokens=235), content="To create rollout resources for the canary and stable services using Kubernetes Gateway API and Argo Rollouts, you will need to apply manifests that define these resources. You will typically need to define a Rollout manifest that includes strategies for canary deployments as well as integration with the Gateway API to manage traffic splitting between versions.\n\nHere's a high-level outline of what needs to be done:\n\n1. **Create Rollout Manifests**: You'll need to create one or more YAML manifests that describe the rollout resources. This includes specifying strategies such as canary or blue-green, and configuration for workload

In [ ]:
print(team.dump_component().model_dump_json(indent=2))